In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/note/LeafDD

/content/drive/MyDrive/note/LeafDD


In [ ]:
!ls datasets/

Cerscospora  Healthy  Miner  Phoma  Rust


In [ ]:
import numpy as np
import tensorflow as tf
import cv2, os
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras.utils.image_utils import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from keras.optimizers import SGD, schedules

In [ ]:
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        # Lưu ảnh tiền xử lý
        self.preprocessors = preprocessors

        # Nếu bước tiền xử lý là None thì khởi tạo danh sách rỗng
        if self.preprocessors is None:
            self.preprocessors = []

    def load(self, imagePaths, verbose=-1):
        # Khởi tạo danh sách các đặc trưng và nhãn
        data = []
        labels = []

        # Lặp qua tất cả ảnh đầu vào
        for (i, imagePath) in enumerate(imagePaths):
            # Nạp ảnh và trích xuất nhãn từ đường dẫn định dạng
            # /path/to/dataset/{class}/{image}.jpg
            image = cv2.imread(imagePath)
            label = imagePath.split(os.path.sep)[-2]
            # check to see if our preprocessors are not None
            if self.preprocessors is not None:
                # Lặp qua tất cả tiền xử lý và áp dụng cho mỗi ảnh
                for p in self.preprocessors:
                    image = p.preprocess(image)
            # Mỗi ảnh được xử lý là vector đặc trưng bằng cách
            # cập nhật danh sách dữ liệu cùng với nhãn
            data.append(image)
            labels.append(label)

            # Hiển thị ảnh cập nhật
            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
               print("[INFO] Đã xử lý {}/{}".format(i + 1,len(imagePaths)))
                # Trả về dữ liệu kiểu tuple gồm dữ liệu và nhãn
        return (np.array(data), np.array(labels))

In [ ]:
class ImageToArrayPreprocessor:  # Tạo lớp để chuyển ảnh --> mảng
    def __init__(self, dataFormat=None):
        # Lưu ảnh đã được định dạng
        self.dataFormat = dataFormat

    def preprocess(self, image): # Định nghĩa phương thức preprocess trả về mảng
        # Hàm img_to_array của Keras
        return img_to_array(image, data_format=self.dataFormat)
class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        # Lưu image width, height và interpolation
        self.width = width
        self.height = height
        self.inter = inter

    def preprocess(self, image):
        # Trả về ảnh có kích thước đã thay đổi
        return cv2.resize(image, (self.width, self.height), interpolation=self.inter)

In [ ]:
class ShallowNet:
    @staticmethod
    def build(width, height, depth, classes):
        # khởi tạo mô hình
        # height, width, depth: tương ứng 3 chiều của dữ liệu ảnh đầu vào
        # classes: tổng số lớp mà mạng dự đoán, phụ thuộc vào dữ liệu
        # Dữ liệu Animal: 3 lớp; Dữ liệu CIFAR-10: 10 lớp

        model = Sequential()
        inputShape = (height, width, depth)

        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)

        # Định nghĩa mạng CONV => RELU layer
        model.add(Conv2D(32, (3, 3), padding="same",input_shape = inputShape))
        model.add(Activation("relu"))

        # Bộ phân lớp sử dụng hàm softmax
        model.add(Flatten())       # Chuyển thành vector
        model.add(Dense(classes))  # Định nghĩa Full Connected layer
        model.add(Activation("softmax"))   # Kích hoạt hàm softmax để phân lớp
        # Trả về model kiến trúc mạng
        return model

In [ ]:
class MiniVGGNet:
  @staticmethod
  def build(width, height, depth, classes):
      # Khởi tạo mô hình, shape ảnh đầu vào và số kênh của ảnh đầu vào
      model = Sequential()
      input_shape = (height, width, depth)
      channel_dim = -1  # chỉ số của số kênh ảnh đầu vào
                        # giá trị -1 ý muốn nói chỉ số kênh nằm cuối cùng
                        # của danh sách chứa dữ liệu ảnh đầu vào

      # sử dụng 'channels_first' để cập nhật shape và số kênh ảnh đầu vào
      if K.image_data_format() == 'channels_first':
          input_shape = (depth, height, width)
          channel_dim = 1

      # Chuỗi layer đầu tiên  CONV => RELU => CONV => RELU => POOL
      model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
      model.add(Activation('relu'))
      model.add(BatchNormalization(axis=channel_dim))
      model.add(Conv2D(32, (3, 3), padding='same'))
      model.add(Activation('relu'))
      model.add(BatchNormalization(axis=channel_dim))
      model.add(MaxPooling2D(pool_size=(2, 2)))
      model.add(Dropout(0.25))

      # Chuỗi layer thứ hai CONV => RELU => CONV => RELU => POOL
      model.add(Conv2D(64, (3, 3), padding='same'))
      model.add(Activation('relu'))
      model.add(BatchNormalization(axis=channel_dim))
      model.add(Conv2D(64, (3, 3), padding='same'))
      model.add(Activation('relu'))
      model.add(BatchNormalization(axis=channel_dim))
      model.add(MaxPooling2D(pool_size=(2, 2)))
      model.add(Dropout(0.25))

      # Thiết lập FC thứ nhất => RELU layers
      model.add(Flatten())
      model.add(Dense(512))
      model.add(Activation('relu'))
      model.add(BatchNormalization())
      model.add(Dropout(0.25))   # Dropout 50%

      # THiết lập FC thứ hai => Hàm phân lớp Softmax
      model.add(Dense(classes))
      model.add(Activation('softmax'))

      # Trả về kiến trúc mạng/mô hình
      return model

In [ ]:
class ModVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        # Khởi tạo mô hình, shape ảnh đầu vào và số kênh của ảnh đầu vào
        model = Sequential()
        input_shape = (height, width, depth)
        channel_dim = -1  # chỉ số của số kênh ảnh đầu vào
                          # giá trị -1 ý muốn nói chỉ số kênh nằm cuối cùng
                          # của danh sách chứa dữ liệu ảnh đầu vào

        # sử dụng 'channels_first' để cập nhật shape và số kênh ảnh đầu vào
        if K.image_data_format() == 'channels_first':
            input_shape = (depth, height, width)
            channel_dim = 1

        # Chuỗi layer đầu tiên  CONV => RELU => CONV => RELU => POOL
        model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(Conv2D(32, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Chuỗi layer thứ hai CONV => RELU => CONV => RELU => POOL
        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Chuỗi layer thứ ba CONV => RELU => CONV => RELU => POOL
        model.add(Conv2D(128, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(Conv2D(128, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Thiết lập FC thứ nhất => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))   # Dropout 50%

        # THiết lập FC thứ hai => Hàm phân lớp Softmax
        model.add(Dense(classes))
        model.add(Activation('softmax'))

        # Trả về kiến trúc mạng/mô hình
        return model

In [ ]:
vb = 20

imagePaths = list(paths.list_images("datasets/"))

# Bước 1. Chuẩn bị dữ liệu
# Khởi tạo tiền xử lý ảnh
sp = SimplePreprocessor(32, 32) # Thiết lập kích thước ảnh 32 x 32
iap = ImageToArrayPreprocessor() # Gọi hàm để chuyển ảnh sang mảng

# Nạp dataset từ đĩa
print("[INFO] Nạp ảnh...")

sdl = SimpleDatasetLoader(preprocessors=[sp, iap])
(data, labels) = sdl.load(imagePaths, verbose=vb)
data = data.astype("float") / 255.0

[INFO] Nạp ảnh...
[INFO] Đã xử lý 20/9120
[INFO] Đã xử lý 40/9120
[INFO] Đã xử lý 60/9120
[INFO] Đã xử lý 80/9120
[INFO] Đã xử lý 100/9120
[INFO] Đã xử lý 120/9120
[INFO] Đã xử lý 140/9120
[INFO] Đã xử lý 160/9120
[INFO] Đã xử lý 180/9120
[INFO] Đã xử lý 200/9120
[INFO] Đã xử lý 220/9120
[INFO] Đã xử lý 240/9120
[INFO] Đã xử lý 260/9120
[INFO] Đã xử lý 280/9120
[INFO] Đã xử lý 300/9120
[INFO] Đã xử lý 320/9120
[INFO] Đã xử lý 340/9120
[INFO] Đã xử lý 360/9120
[INFO] Đã xử lý 380/9120
[INFO] Đã xử lý 400/9120
[INFO] Đã xử lý 420/9120
[INFO] Đã xử lý 440/9120
[INFO] Đã xử lý 460/9120
[INFO] Đã xử lý 480/9120
[INFO] Đã xử lý 500/9120
[INFO] Đã xử lý 520/9120
[INFO] Đã xử lý 540/9120
[INFO] Đã xử lý 560/9120
[INFO] Đã xử lý 580/9120
[INFO] Đã xử lý 600/9120
[INFO] Đã xử lý 620/9120
[INFO] Đã xử lý 640/9120
[INFO] Đã xử lý 660/9120
[INFO] Đã xử lý 680/9120
[INFO] Đã xử lý 700/9120
[INFO] Đã xử lý 720/9120
[INFO] Đã xử lý 740/9120
[INFO] Đã xử lý 760/9120
[INFO] Đã xử lý 780/9120
[INFO] Đã x

In [ ]:
classLabels = ["Cerscospora", "Healthy", "Miner", "Phoma", "Rust"]

# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=1,)
optimizer = SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True)

In [ ]:
model = AlexNet.build(width=32, height=32, depth=3, classes=5)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=30, verbose=1)

model.save("miniVGGNet.hdf5")
# model.summary()

print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classLabels))

In [ ]:
model = ShallowNet.build(width=32, height=32, depth=3, classes=5)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=30, verbose=1)

model.save("ShallowNet.hdf5")
# model.summary()

print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classLabels))

[INFO]: Đang trainning....
Epoch 1/30
107/107 [==============================] - 3s 17ms/step - loss: 1.3303 - accuracy: 0.4958 - val_loss: 0.7334 - val_accuracy: 0.7075
Epoch 2/30
107/107 [==============================] - 1s 11ms/step - loss: 0.5570 - accuracy: 0.7765 - val_loss: 0.4874 - val_accuracy: 0.7982
Epoch 3/30
107/107 [==============================] - 1s 7ms/step - loss: 0.3817 - accuracy: 0.8564 - val_loss: 0.3444 - val_accuracy: 0.8697
Epoch 4/30
107/107 [==============================] - 1s 6ms/step - loss: 0.2960 - accuracy: 0.9003 - val_loss: 0.3091 - val_accuracy: 0.9013
Epoch 5/30
107/107 [==============================] - 1s 6ms/step - loss: 0.2548 - accuracy: 0.9118 - val_loss: 0.3882 - val_accuracy: 0.8623
Epoch 6/30
107/107 [==============================] - 1s 6ms/step - loss: 0.2308 - accuracy: 0.9225 - val_loss: 0.3030 - val_accuracy: 0.8842
Epoch 7/30
107/107 [==============================] - 1s 6ms/step - loss: 0.1950 - accuracy: 0.9352 - val_loss: 0.2406 

In [ ]:
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=5)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=30, verbose=1)

model.save("miniVGGNet.hdf5")
# model.summary()

print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classLabels))

[INFO]: Đang trainning....
Epoch 1/30
107/107 [==============================] - 3s 16ms/step - loss: 0.4509 - accuracy: 0.8402 - val_loss: 1.8851 - val_accuracy: 0.4675
Epoch 2/30
107/107 [==============================] - 1s 11ms/step - loss: 0.1276 - accuracy: 0.9538 - val_loss: 1.0038 - val_accuracy: 0.6833
Epoch 3/30
107/107 [==============================] - 1s 12ms/step - loss: 0.0890 - accuracy: 0.9686 - val_loss: 0.8404 - val_accuracy: 0.7645
Epoch 4/30
107/107 [==============================] - 1s 12ms/step - loss: 0.0529 - accuracy: 0.9844 - val_loss: 0.0679 - val_accuracy: 0.9789
Epoch 5/30
107/107 [==============================] - 1s 12ms/step - loss: 0.0476 - accuracy: 0.9832 - val_loss: 0.0825 - val_accuracy: 0.9781
Epoch 6/30
107/107 [==============================] - 1s 13ms/step - loss: 0.0341 - accuracy: 0.9892 - val_loss: 0.0582 - val_accuracy: 0.9820
Epoch 7/30
107/107 [==============================] - 1s 12ms/step - loss: 0.0343 - accuracy: 0.9880 - val_loss: 0.

In [ ]:
model = ModVGGNet.build(width=32, height=32, depth=3, classes=5)
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=30, verbose=1)

model.save("modVGGNet.hdf5")
# model.summary()

print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classLabels))

[INFO]: Đang trainning....
Epoch 1/30
107/107 [==============================] - 3s 16ms/step - loss: 0.6029 - accuracy: 0.7958 - val_loss: 2.8798 - val_accuracy: 0.2632
Epoch 2/30
107/107 [==============================] - 1s 14ms/step - loss: 0.2053 - accuracy: 0.9273 - val_loss: 1.1952 - val_accuracy: 0.6421
Epoch 3/30
107/107 [==============================] - 1s 13ms/step - loss: 0.1291 - accuracy: 0.9522 - val_loss: 0.6774 - val_accuracy: 0.8443
Epoch 4/30
107/107 [==============================] - 1s 13ms/step - loss: 0.0983 - accuracy: 0.9689 - val_loss: 0.3490 - val_accuracy: 0.9127
Epoch 5/30
107/107 [==============================] - 1s 13ms/step - loss: 0.0738 - accuracy: 0.9741 - val_loss: 0.0648 - val_accuracy: 0.9794
Epoch 6/30
107/107 [==============================] - 2s 14ms/step - loss: 0.0622 - accuracy: 0.9781 - val_loss: 0.0484 - val_accuracy: 0.9833
Epoch 7/30
107/107 [==============================] - 2s 16ms/step - loss: 0.0448 - accuracy: 0.9849 - val_loss: 0.